### Short-term memory

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # [get_user_info],
    # checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='efdc3c67-5311-41d6-ab8b-a7c1150314d3'),
  AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--00552fea-f46f-4352-a0ab-5525aec9171e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 18, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})]}

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐였지?"}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='내 이름이 뭐였지?', additional_kwargs={}, response_metadata={}, id='602eeb32-4b25-4022-bcc4-eb17c16ce8d5'),
  AIMessage(content='죄송합니다. 저는 대화 기록을 저장하지 않기 때문에 이전에 제가 당신의 이름을 알았는지, 혹은 당신의 이름이 무엇인지 알 수 없습니다.\n\n혹시 다시 알려주시겠어요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--90757089-5c1f-4ae7-9fdb-c52c7f7dc211-0', usage_metadata={'input_tokens': 8, 'output_tokens': 44, 'total_tokens': 52, 'input_token_details': {'cache_read': 0}})]}

에이전트에 단기 메모리(스레드 수준 지속성)를 추가하려면 에이전트를 생성할 때 checkpointer 지정해야 합니다.

In [3]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # [get_user_info],
    checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='b08bcb81-5bb0-4ac4-871f-62add6192b3b'),
  AIMessage(content="Hi Bob! It's nice to meet you. I'm a large language model, trained by Google. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dd0bebcb-aa4a-4201-b530-3f74f718486b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 30, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})]}

In [4]:
agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐였지?"}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='b08bcb81-5bb0-4ac4-871f-62add6192b3b'),
  AIMessage(content="Hi Bob! It's nice to meet you. I'm a large language model, trained by Google. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dd0bebcb-aa4a-4201-b530-3f74f718486b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 30, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='내 이름이 뭐였지?', additional_kwargs={}, response_metadata={}, id='1206530c-93e0-4904-ae86-24247e84258f'),
  AIMessage(content='당신의 이름은 Bob이었습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_nam